# Computational Module Theory
## by Vladislav Sliusarev

In my project, I've implemented algorithms for solving several problems from the Homological Algebra and Module Theory course in a Python library.

The code is available at https://github.com/v-slyusarev/computational_module_theory.

This notebook is available at https://github.com/v-slyusarev/computational_module_theory/blob/master/Computational%20Module%20Theory.ipynb.

This notebook overviews the functions I've implemented.

## Table of contents
* [Introduction](#Introduction)
* [Direct sum of modules](#Direct-sum-of-modules)
* [Tensor product of modules](#Tensor-product-of-modules)
* [$Hom$](#$Hom$)
* [Submodules and quotients](#Submodules-and-quotients)
* [Kernels of homomorphisms](#Kernels-of-homomorphisms)
* [Cohomology](#Cohomology)

## Introduction
In this project we work with finitely generated $\mathbb{Z}$-modules. Since $\mathbb{Z}$ is a PID, every such module $A$ can be represented as a direct sum of cyclic modules:
$$
A = \mathbb{Z} \oplus ... \mathbb{Z} \oplus \mathbb{Z}/q_1\mathbb{Z} \oplus ... \mathbb{Z}/q_m\mathbb{Z},
$$
where $m \ge 0$ and $q_1, \ldots, q_m \ge 2$. We will call it the *standard representation* of a finitely generated $\mathbb{Z}$-module.

**Note.** The Fundamental Theorem provides such representation with $q_1 \mid \ldots \mid q_m$, and such representation is unique. However, in our computations neither divisibility nor uniqueness are crucial, while ensuring this requirement at any step complicates the computations. Therefore we will not impose such restrictions and work with arbitrary direct sums of cyclic modules. For example, 
$$
\mathbb{Z} \oplus \mathbb{Z}/2\mathbb{Z} \oplus \mathbb{Z}/ 3\mathbb{Z},\quad\mathbb{Z} \oplus \mathbb{Z}/3\mathbb{Z} \oplus \mathbb{Z}/ 2\mathbb{Z},\quad\mathbb{Z} \oplus \mathbb{Z}/6\mathbb{Z}
$$
are all valid standard representations of the same $\mathbb{Z}$-module in this project. 

A standard representation is given by two parameters:
* A non-negative integer $n$;
* A finite (possibly empty) sequence of integers $q_1,\, \ldots,\, q_m \ge 2$.

In the next cell you can find an example of defining a finitely generating $\mathbb{Z}$-module in our library.

In [1]:
from module_theory.zmodule import ZModule

A = ZModule(2, [2, 3])
print(A)

ℤ^2 ⊕ ℤ/2ℤ ⊕ ℤ/3ℤ


An element of a module is represented by its coordinates — a sequence of $n+m$ integers $x_1, ..., x_{n+m}$, where $0 \le x_{n+j} < q_j$.

After we defined a module, we can work with its elements as shown below.

In [2]:
x = A.element([1, 0, 1, 1])
y = A.element([0, 1, 1, -1])
print(x + y)
print(3 * x)

(1, 1, 0 + 2ℤ, 0 + 3ℤ)
(3, 0, 1 + 2ℤ, 0 + 3ℤ)


A $\mathbb{Z}$-module homomorphism is represented by a matrix. If $f: A \to B$ is a homomorphism, where
$$
A = \mathbb{Z}^n \oplus \mathbb{Z}/p_1\mathbb{Z} \oplus ... \oplus \mathbb{Z}/p_m\mathbb{Z},
$$
$$
B = \mathbb{Z}^k \oplus \mathbb{Z}/q_1\mathbb{Z} \oplus ... \oplus \mathbb{Z}/q_l\mathbb{Z},
$$
then its matrix is a $(k+l)\times (n+m)$-matrix of integers
$$
\begin{pmatrix}
a_{1,1} & a_{1,2} & \cdots & a_{1, n+m}
\\
\vdots & \vdots & \ddots & \vdots
\\
a_{k+l,1} & a_{k+l,2} & \cdots & a_{k+l, n+m}
\end{pmatrix},
$$
where
$$
\begin{align}
&(a_{1,1},\, a_{1,2},\, \ldots,\, a_{1, k+l}) &&= && f(1, 0, \ldots, 0),
\\
&(a_{2,1},\, a_{2,2},\, \ldots,\, a_{2, k+l}) &&= && f(1, 0, \ldots, 0),
\\
&(a_{n+m,1},\, a_{n+m,2},\, \ldots,\, a_{n+m, k+l}) &&=&& f(0, 0, \ldots, 1).
\end{align}
$$

**Note.** The correspondence between the homomorphisms and the matrices is not bijective. For example, $\begin{pmatrix}1\end{pmatrix}$ does not represent a homomorphism from $\mathbb{Z} / 2\mathbb{Z}$ to $\mathbb{Z}$. However, any homomorphism is represented by some matrix.

If $F$ is a matrix that represents a homomorphism $f: A \to B$, and $x = (x_1, \ldots, x_{n+m}) \in A$, then $f(x) = Fx$, where the multiplication is defined an the standard way.

If $F$ and $G$ are matrices of homomorphisms $f: A \to B$ and $g: B \to C$, then the composition $g \circ f$ is represented by the matrix $GF$.

Then next cell shows how we work with homomorphisms.

In [3]:
from module_theory.homomorphism import Homomorphism

A = ZModule(2, [2, 3])
B = ZModule(1, [2])
C = ZModule(1,[2,2])
f = Homomorphism(
    [[2, 0, 1, 0],
     [0, -3, 0, 0]],
    domain=A, codomain=B
)
print(f)

ℤ^2 ⊕ ℤ/2ℤ ⊕ ℤ/3ℤ --> ℤ ⊕ ℤ/2ℤ,
(2, 0, 1, 0)
(0, 1, 0, 0)


In [4]:
g = Homomorphism(
    [[1, 0],
     [0, 0],
     [0, 1]],
    domain=B, codomain=C
)
print(g)

ℤ ⊕ ℤ/2ℤ --> ℤ ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ,
(1, 0)
(0, 0)
(0, 1)


In [5]:
x = A.element([2, -1, 1, -2])
print(f.apply(x))

(5, 1 + 2ℤ)


In [6]:
print(g.compose(f))

ℤ^2 ⊕ ℤ/2ℤ ⊕ ℤ/3ℤ --> ℤ ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ,
(2, 0, 1, 0)
(0, 0, 0, 0)
(0, 1, 0, 0)


A cochain complex is given by a list of modules and a list of homomorphisms.

In [7]:
from module_theory.cochain_complex import CochainComplex

A = ZModule(1, [])
B = ZModule(1, [2])
C = ZModule(0, [2])

f = Homomorphism(
    [[1],
     [0]],
    domain=A, codomain=B
)

g = Homomorphism(
    [[0, 1]],
    domain=B, codomain=C
)


cochain_complex = CochainComplex(
    [A, B, C],
    [Homomorphism.zero(A, B), Homomorphism.zero(B, C)]
)
print(cochain_complex)

0 --d0--> ℤ --d1--> ℤ ⊕ ℤ/2ℤ --d2--> ℤ/2ℤ --d3--> 0
d1: ℤ --> ℤ ⊕ ℤ/2ℤ,
(0,)
(0,)
d2: ℤ ⊕ ℤ/2ℤ --> ℤ/2ℤ,
(0, 0)
d3: ℤ/2ℤ --> 0,
(0,)



## Direct sum of modules
Given two $\mathbb{Z}$-modules
$$
A = \mathbb{Z}^n \oplus \mathbb{Z}/p_1\mathbb{Z} \oplus ... \oplus \mathbb{Z}/p_m\mathbb{Z},
$$
$$
B = \mathbb{Z}^k \oplus \mathbb{Z}/q_1\mathbb{Z} \oplus ... \oplus \mathbb{Z}/q_l\mathbb{Z},
$$
the direct sum $A \oplus B$ has a standard representation
$$
\mathbb{Z}^{n+k} \oplus \mathbb{Z}/p_1\mathbb{Z} \oplus ... \oplus \mathbb{Z}/p_m\mathbb{Z} \oplus \mathbb{Z}/q_1\mathbb{Z} \oplus ... \oplus \mathbb{Z}/q_l\mathbb{Z}.
$$
Similarly, we can calculate an arbitrary direct sum $\bigoplus_{i=1}^k A_i$. For every summand $A_i$, there is an embedding $f_i: A_i \to \bigoplus_{i=1}^k A_i$, whose matrix is an identity matrix padded with zero entries.

In [8]:
from module_theory.operators.direct_sum import direct_sum

A = ZModule(0, [2])
B = ZModule(2, [])
C = ZModule(1, [2, 3])

print("A = ", A)
print("B = ", B)
print("C = ", C)
print()

sum_module, embeddings = direct_sum(A, B, C)
f = embeddings[0]
g = embeddings[1]
h = embeddings[2]

print("A ⊕ B ⊕ C ≅ ", sum_module)
print("A embeds by f:", f)
print("B embeds by g:", g)
print("C embeds by h:", h)


A =  ℤ/2ℤ
B =  ℤ^2
C =  ℤ ⊕ ℤ/2ℤ ⊕ ℤ/3ℤ

A ⊕ B ⊕ C ≅  ℤ^3 ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ ⊕ ℤ/3ℤ
A embeds by f: ℤ/2ℤ --> ℤ^3 ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ ⊕ ℤ/3ℤ,
(0,)
(0,)
(0,)
(1,)
(0,)
(0,)
B embeds by g: ℤ^2 --> ℤ^3 ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ ⊕ ℤ/3ℤ,
(1, 0)
(0, 1)
(0, 0)
(0, 0)
(0, 0)
(0, 0)
C embeds by h: ℤ ⊕ ℤ/2ℤ ⊕ ℤ/3ℤ --> ℤ^3 ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ ⊕ ℤ/3ℤ,
(0, 0, 0)
(0, 0, 0)
(1, 0, 0)
(0, 0, 0)
(0, 1, 0)
(0, 0, 1)


## Tensor product of modules
Since the tensor product distributes over direct sums, we can readily calculate a standard representation of $A \otimes B$ given standard representations of $A$ and $B$. Indeed,
$$
\left(\bigoplus_{i=1}^n A_i\right) \otimes \left(\bigoplus_{j=1}^m B_i\right) = \bigoplus_{i=1}^n \bigoplus_{j=1}^m A_i \otimes B_j.
$$
Notice that
$$
\begin{align}
\mathbb{Z} \otimes \mathbb{Z} &\cong \mathbb{Z};
\\
\mathbb{Z} \otimes \mathbb{Z} / q\mathbb{Z} &\cong \mathbb{Z} / q\mathbb{Z};
\\
\mathbb{Z} / q\mathbb{Z} \otimes \mathbb{Z} &\cong \mathbb{Z} / q\mathbb{Z};
\\
\mathbb{Z} / p\mathbb{Z} \otimes \mathbb{Z} / q\mathbb{Z} &\cong \mathbb{Z} / \gcd(p,q)\mathbb{Z}.
\end{align}
$$
Then we can use the direct sum algorithm that we already implemented to find standard representations and embeddings of direct summands for tensor products. The embeddings allow us to calculate the coordinates for tensor products of elements and the matrices for tensor products of homomorphisms.

The same applies for tensor products of an arbitrary finite number of modules.

In [9]:
from module_theory.operators.tensor_product import TensorProduct

A = ZModule(1, [6])
B = ZModule(1, [8])
C = TensorProduct(A, B)

print(C)

ℤ ⊕ ℤ/8ℤ ⊕ ℤ/6ℤ ⊕ ℤ/2ℤ


In [10]:
a = A.element((-10, 9))
b = B.element((20, 9))
c = A.element((1, 1))
d =  B.element((1, 1))
a_times_b = C.pure_tensor(a, b)
c_times_d = C.pure_tensor(c, d)
print("a = ", a)
print("b = ", b)
print("a ⊗ b = ", a_times_b)
print("c ⊗ d = ", c_times_d)
print("a ⊗ b + c ⊗ d = ", a_times_b + c_times_d)

a =  (-10, 3 + 6ℤ)
b =  (20, 1 + 8ℤ)
a ⊗ b =  (-200, 6 + 8ℤ, 0 + 6ℤ, 1 + 2ℤ)
c ⊗ d =  (1, 1 + 8ℤ, 1 + 6ℤ, 1 + 2ℤ)
a ⊗ b + c ⊗ d =  (-199, 7 + 8ℤ, 1 + 6ℤ, 0 + 2ℤ)


In [11]:
D = ZModule(1, [48])
f = Homomorphism([
    [1, 2],
     [3, 4]],
    domain=A, codomain=D
)

g = Homomorphism(
    [[5, 6],
     [7, 8]],
    domain=B, codomain=D)

f_times_g = C.homomorphism(f, g)
print("f:", f)
print("g:", g)
print("f ⊗ g:", f_times_g)
print("(f ⊗ g)(a ⊗ b + c ⊗ d) = ", f_times_g.apply(a_times_b + c_times_d))

f: ℤ ⊕ ℤ/6ℤ --> ℤ ⊕ ℤ/48ℤ,
(1, 2)
(3, 4)
g: ℤ ⊕ ℤ/8ℤ --> ℤ ⊕ ℤ/48ℤ,
(5, 6)
(7, 8)
f ⊗ g: ℤ ⊕ ℤ/8ℤ ⊕ ℤ/6ℤ ⊕ ℤ/2ℤ --> ℤ ⊕ ℤ/48ℤ ⊕ ℤ/48ℤ ⊕ ℤ/48ℤ,
(5, 6, 10, 12)
(7, 8, 14, 16)
(15, 18, 20, 24)
(21, 24, 28, 32)
(f ⊗ g)(a ⊗ b + c ⊗ d) =  (-943, 21 + 48ℤ, 41 + 48ℤ, 1 + 48ℤ)


We can use the tensor products of homomorphisms to apply the $A \otimes -$ and $- \otimes B$ functors to cochain complexes.

In the example below, we apply $\mathbb{Z} / 2 \mathbb{Z} \otimes -$ to the cochain complex
$$
0 \longrightarrow \mathbb{Z} \oplus \mathbb{Z} / 4\mathbb{Z} \longrightarrow \mathbb{Z}^2 \oplus \mathbb{Z} / 2\mathbb{Z} \longrightarrow \mathbb{Z} \oplus \mathbb{Z} / 4\mathbb{Z} \longrightarrow 0
$$
with chain maps given by $
\begin{pmatrix}
1&  0
\\
2&  0
\\
1&  1
\end{pmatrix}
$ and $
\begin{pmatrix}
-2 & 1 & 0
\\
0 & 0 & 0
\end{pmatrix}
$.

In [12]:
C0 = ZModule(1, [4])
C1 = ZModule(2, [2])
C2 = ZModule(1, [4])
d1 = Homomorphism((
    (1, 0),
    (2, 0),
    (1, 1),
), C0, C1)
d2 = Homomorphism((
    (-2, 1, 0),
    (0, 0, 0),
), C1, C2)
cochain_complex = CochainComplex(
    modules=[C0, C1, C2],
    homomorphisms=[d1, d2]
)

print(cochain_complex)

0 --d0--> ℤ ⊕ ℤ/4ℤ --d1--> ℤ^2 ⊕ ℤ/2ℤ --d2--> ℤ ⊕ ℤ/4ℤ --d3--> 0
d1: ℤ ⊕ ℤ/4ℤ --> ℤ^2 ⊕ ℤ/2ℤ,
(1, 0)
(2, 0)
(1, 1)
d2: ℤ^2 ⊕ ℤ/2ℤ --> ℤ ⊕ ℤ/4ℤ,
(-2, 1, 0)
(0, 0, 0)
d3: ℤ ⊕ ℤ/4ℤ --> 0,
(0, 0)



In [13]:
from module_theory.operators.tensor_product import left_tensor_product

A = ZModule(0, [2])
result = left_tensor_product(cochain_complex, A)
print(result)

0 --d0--> ℤ/2ℤ ⊕ ℤ/2ℤ --d1--> ℤ/2ℤ ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ --d2--> ℤ/2ℤ ⊕ ℤ/2ℤ --d3--> 0
d1: ℤ/2ℤ ⊕ ℤ/2ℤ --> ℤ/2ℤ ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ,
(1, 0)
(0, 0)
(1, 1)
d2: ℤ/2ℤ ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ --> ℤ/2ℤ ⊕ ℤ/2ℤ,
(0, 1, 0)
(0, 0, 0)
d3: ℤ/2ℤ ⊕ ℤ/2ℤ --> 0,
(0, 0)



## $Hom$

Given $\mathbb{Z}$-modules $A$ and $B$, the set of all homomorphisms from $A$ to $B$ forms a \mathbb{Z}-module, which we denote $Hom(A, B)$.

Since we work with finitely generated modules, the direct product and the direct sum coincide, therefore $Hom$ distributes over direct sums:
$$
Hom\left(\bigoplus_{i=1}^n A_i, \bigoplus_{j=1}^n B_j\right) \cong \bigoplus_{i=1}^n \bigoplus_{j=1}^n Hom(A_i, B_j).
$$
Then we can use the following isomorphisms to calculate standard representations for $Hom(A, B)$ for any finitely generated $\mathbb{Z}$-modules $A$ and $B$:
$$
\begin{align}
Hom(\mathbb{Z}, B) &\cong B;
\\
Hom(\mathbb{Z} / p\mathbb{Z}, \mathbb{Z}) &\cong 0;
\\
Hom(\mathbb{Z} / p \mathbb{Z},  \mathbb{Z} / q \mathbb{Z}) &\cong \mathbb{Z} / gcd(p,q) \mathbb{Z}.
\end{align}
$$
Again, we use these equations and the direct sum algorithm that we implemented above to find standard representations of $Hom(A, B)$. Using the direct summand embeddings, we can convert between matrices and coordinates in $Hom(A, B)$.

In [14]:
from module_theory.operators.hom import Hom

A = ZModule(1, [2, 4])
B = ZModule(2, [5, 10])
homAB = Hom(A, B)
print(f"Hom({A}, {B}) ≅ {homAB})")

Hom(ℤ ⊕ ℤ/2ℤ ⊕ ℤ/4ℤ, ℤ^2 ⊕ ℤ/5ℤ ⊕ ℤ/10ℤ) ≅ ℤ^2 ⊕ ℤ/5ℤ ⊕ ℤ/10ℤ ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ)


In [15]:
matrix = ((1, 0, 0),
          (2, 0, 0),
          (3, 0, 0),
          (4, 1, 1))
f = Homomorphism(matrix, A, B)

print(f)

x = homAB.element_from_homomorphism(f)
print(x)

ℤ ⊕ ℤ/2ℤ ⊕ ℤ/4ℤ --> ℤ^2 ⊕ ℤ/5ℤ ⊕ ℤ/10ℤ,
(1, 0, 0)
(2, 0, 0)
(3, 0, 0)
(4, 1, 1)
(1, 2, 3 + 5ℤ, 4 + 10ℤ, 1 + 2ℤ, 1 + 2ℤ)


In [16]:
print(homAB.homomorphism_from_element(x))

ℤ ⊕ ℤ/2ℤ ⊕ ℤ/4ℤ --> ℤ^2 ⊕ ℤ/5ℤ ⊕ ℤ/10ℤ,
(1, 0, 0)
(2, 0, 0)
(3, 0, 0)
(4, 1, 1)


Using these algorithms, we can apply $Hom(A, -)$ and $Hom(-, B)$ to cochain complexes. In the example below, we apply $Hom(\mathbb{Z}^2, -)$ to the split exact sequence
$$
0 \longrightarrow \mathbb{Z} / 3 \mathbb{Z} \longrightarrow \mathbb{Z} / 3 \mathbb{Z} \oplus \mathbb{Z} / 6 \mathbb{Z} \longrightarrow \mathbb{Z} / 6 \mathbb{Z} \longrightarrow 0.
$$

In [17]:
from module_theory.operators.hom import left_hom

A = ZModule(0, [3])
C = ZModule(0, [6])
B, embeddings = direct_sum(A, C)
cochain_complex = CochainComplex(
    modules=[A, B, C],
    homomorphisms=[
        embeddings[0],
        Homomorphism([[0, 1]], domain=B, codomain=C)
    ]
)

D = ZModule.free(2)
hom_cochain_complex = left_hom(cochain_complex, D)

print(hom_cochain_complex)

0 --d0--> ℤ/3ℤ ⊕ ℤ/3ℤ --d1--> ℤ/3ℤ ⊕ ℤ/3ℤ ⊕ ℤ/6ℤ ⊕ ℤ/6ℤ --d2--> ℤ/6ℤ ⊕ ℤ/6ℤ --d3--> 0
d1: ℤ/3ℤ ⊕ ℤ/3ℤ --> ℤ/3ℤ ⊕ ℤ/3ℤ ⊕ ℤ/6ℤ ⊕ ℤ/6ℤ,
(1, 0)
(0, 1)
(0, 0)
(0, 0)
d2: ℤ/3ℤ ⊕ ℤ/3ℤ ⊕ ℤ/6ℤ ⊕ ℤ/6ℤ --> ℤ/6ℤ ⊕ ℤ/6ℤ,
(0, 0, 1, 0)
(0, 0, 0, 1)
d3: ℤ/6ℤ ⊕ ℤ/6ℤ --> 0,
(0, 0)



In the next example, we apply $Hom(-, \mathbb{Z} / 2 \mathbb{Z})$ to the split exact sequence
$$
0 \longrightarrow \mathbb{Z}^2 \longrightarrow \mathbb{Z}^3 \longrightarrow \mathbb{Z} \longrightarrow 0.
$$

In [18]:
from module_theory.operators.hom import right_hom

A = ZModule.free(2)
C = ZModule.free(1)
B, embeddings = direct_sum(A, C)

cochain_complex = CochainComplex(
    modules=[A, B, C],
    homomorphisms=[
        embeddings[0],
        Homomorphism(
            matrix=[[0, 0, 1]],
            domain=B,
            codomain=C)
    ]
)

D = ZModule(0, [2])
hom_cochain_complex = right_hom(cochain_complex, D)
print(hom_cochain_complex)

0 --d0--> ℤ/2ℤ --d1--> ℤ/2ℤ ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ --d2--> ℤ/2ℤ ⊕ ℤ/2ℤ --d3--> 0
d1: ℤ/2ℤ --> ℤ/2ℤ ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ,
(0,)
(0,)
(1,)
d2: ℤ/2ℤ ⊕ ℤ/2ℤ ⊕ ℤ/2ℤ --> ℤ/2ℤ ⊕ ℤ/2ℤ,
(1, 0, 0)
(0, 1, 0)
d3: ℤ/2ℤ ⊕ ℤ/2ℤ --> 0,
(0, 0)



## Submodules and quotients
And now the tasty part of it. In the remaining part of this notebook, we will introduce an algorithm that calculates the cohomology of a cochain complex. 

We use the definition: $H^n(\mathcal{C}) = \frac{\ker d_{n+1}}{\operatorname{Im} d_{n}}$. Then we need to be able to calculate arbitrary quotients of submodules.

Our first step will be calculating the preimage of a given element under a homomorphism.
Let $F$ be a matrix of a homomorphism $f: A \to B$ and let $b \in B$.
Let us find some $a \in A$ such that $f(a) = b$.
We use the Smith normal form.

**Theorem.** Let $A, B$ be finitely generated $\mathbb{Z}$-modules and let $f: A \to B$ be a homomorphism with a matrix $F$. There exists an algorithm that produces matrices $G,\,Q,\,Q',\,R,\,R'$ such that
$$
G = \left(\begin{array}{c|c}
\begin{matrix}
b_1 & 0 & \ldots & 0
\\
0 & b_2 & \ldots & 0
\\
\vdots & \vdots & \ddots & \vdots
\\
0 & 0 & \ldots & b_k
\end{matrix} & 0
\\
\hline
0 & \begin{matrix}
0 & \ldots & 0
\\
\vdots & \ddots & \vdots
\\
0 & \ldots & 0
\end{matrix}
\end{array}\right),
$$
$Q$ and $R$ define isomorphisms, $Q$ and $Q'$ are mutually inverse, $R$ and $R'$ are mutually inverse, and
$$
G = Q' F R.
$$
$G$ is called the *Smith normal form* of $F$.

By this theorem, solving $Fx = b$ reduces to solving $G R' x = Q'b$. Let $c = Q'b$ and $u = R'x$. The equation $Gu = c$ has a solution iff $b_i$ divides $c_i$ for $1 \le i \le k$ and $c_i = 0$ for $i > k$.

We proceed by describing how to find a quotient $A / (a_1, \ldots, a_n)$. It is easy to produce a homomorphism onto $(a_1, \ldots, a_n)$: just take the matrix $H$ with $n$ rows where the coordinates of $a_i$ constitute the $i^{\text{th}}$ column. Let $G$ be the Smith normal form of $H$. Then:
* Each diagonal entry in $G$ equal to $0$ represents a generator $g$ of $A$ such that $(g) \cap (a_1,\ldots,a_n)=\{0\}$. These elements generate direct summands of the quotient with the same torsion.
* Each diagonal entry in $G$ equal to $1$ represents a generator of $A$ that belongs to $(a_1,\ldots,a_n)$. These elements are killed off by the quotient.
* The rest diagonal entries represent generators $g$ of $A$ such that $k g \in (a_1,\ldots,a_n)$ for some $k \ge 2$ and no smaller $k\in \mathbb{N}$. Such an element generates a direct summand of the quotient with torsion $k$.

Then we obtain a standard representation of $A / (a_1, \ldots, a_n)$.

To find a standard representation of $(w_1, \ldots, w_m) / (v_1, \ldots, v_n)$, we need to modify the algorithm slightly. First, we reduce the list $(w_1, \ldots, w_m)$ using the standard row echelon algorithm to ensure that no sublist of $(w_1, \ldots, w_m)$ spans the same submodule. For $1 \le 1 \le n$, we find a solution to $Wx = v_i$, where $W$ is the matrix that represents the projection of $A$ onto $(w_1, \ldots, w_m)$. Such a solution is given by the Smith normal form of $W$. We get the coordinates of $v_1, \ldots, v_n$ in the basis of the submodule $(w_1, \ldots, w_m)$. Then we apply the procedure described above to find a standard representation of the quotient.

The next cell shows the usage of the submodule-quotient algorithm. Let $A = \mathbb{Z}^4$ and let
$$
B = \{ (0, x_2, x_3, x_4) \mid x_2,\,x_3,\,x_4 \in \mathbb{Z}\} \subseteq A.
$$
Let  $a = (0, 3, 0, 2)$ and $b = (0, 2, 2, 2)$.
Let us find the standard representations for $B$, $A / (a, b)$, and $B / (a, b)$.

In [19]:
from module_theory.operators.submodule_quotient import SubmoduleQuotient, Submodule, Quotient

A = ZModule.free(4)
g1 = A.element([0, 1, 0, 0])
g2 = A.element([0, 0, 1, 0])
g3 = A.element([0, 0, 0, 1])
a = A.element([0, 3, 0, 2])
b = A.element([0, 2, 2, 2])

In [20]:
B = Submodule([g1, g2, g3])
print(B)

ℤ^3


In [21]:
A_over_a_b = Quotient(A, [a, b])
print(A_over_a_b)

ℤ^2 ⊕ ℤ/2ℤ


In [22]:
B_over_a_b = SubmoduleQuotient([g1, g2, g3], [a, b])
print(B_over_a_b)

ℤ ⊕ ℤ/2ℤ


## Kernels of homomorphisms
Given the matrix $F$ of a homomorphism $f: \mathbb{Z}^m \to \mathbb{Z}^n$ of torsion-free modules, we can find the generators of $\ker f$ using the standard algorithm from linear algebra by finding the row echelon form of $F^T$. In general, the problem of finding the kernel of a homomorphism $f: A \to B$, where 
$$
A = \mathbb{Z}^n \oplus \mathbb{Z}/p_1\mathbb{Z} \oplus ... \oplus \mathbb{Z}/p_m\mathbb{Z},
$$
$$
B = \mathbb{Z}^k \oplus \mathbb{Z}/q_1\mathbb{Z} \oplus ... \oplus \mathbb{Z}/q_l\mathbb{Z},
$$
reduces to the torsion-free case using the following idea. Observe that the following are equivalent:
* $Fx = 0$ in $B$
* $Fx = (y_1, \ldots, y_{k+l})$ such that
$$
\begin{cases}
y_1 = 0,
\\
\cdots
\\
y_k = 0,
\\
\cdots
\\
y_{k+1} = q_1 c_1,
\\
\cdots
\\
y_{k+l} = q_l c_l
\end{cases}
$$
for some $c_1,\,\ldots,\,c_l\in \mathbb{Z}$ 
* $Fx = (y_1, \ldots, y_{k+l})$
for some $z = (y_1, \ldots, y_{k+l}, c_1, \ldots, c_l) \in \mathbb{Z}^{k + 2l}$ that solves an integer system $Hz = 0$, where
$$
H = \left(\begin{array}{c|c}
F & \begin{array}{cccc}
&&0_{k\times l}&&
\\
\hline
-q_1 & 0 & \cdots & 0
\\
0 & -q_2 & \cdots & 0
\\
\vdots & \vdots & \ddots & \vdots
\\
0 & 0 & \cdots & -q_l
\\
\end{array}
\end{array}\right).
$$

In [23]:
A = ZModule.free(4)
B = ZModule(1, [2, 2, 4])
f = Homomorphism(
    ((1, 0, 0, 0),
     (0, 1, 0, 0),
     (0, 0, 1, 0),
     (0, 0, 0, 1)),
    domain=A, codomain=B)
kernel_generators = f.kernel_generators()
print(kernel_generators)

[(0, 2, 0, 0), (0, 0, 2, 0), (0, 0, 0, 4)]


In [24]:
A = ZModule(0, [4, 6])
B = ZModule(0, [2, 6])
f = Homomorphism((
    (1, 1),
    (0, 2)
), A, B)
kernel_generators = f.kernel_generators()
print(kernel_generators)

[(1 + 4ℤ, 3 + 6ℤ)]


## Cohomology
Finally, we are ready to calculate $H^n(\mathcal{C}) = \frac{\ker d_{n+1}}{\operatorname{Im} d_{n}}$.
We already described how to find the generators of $\ker d_{n+1}$. The generators of $\operatorname{Im} d_{n}$ are simply the columns of the matrix that represents $d_n$. Given the generators of the kernel and the image, we apply the submodule-quotient algorithm to find the cohomology group.

In [25]:
from module_theory.cohomology import cohomology

Zsquare = ZModule.free(2)
cochain_complex = CochainComplex(
    modules=[Zsquare, Zsquare],
    homomorphisms=[Homomorphism([
        [1, 2],
        [0, 2]
    ], Zsquare, Zsquare)]
)
print(cochain_complex)
H = cohomology(cochain_complex)
print("The cohomology groups are:")
print(H)

0 --d0--> ℤ^2 --d1--> ℤ^2 --d2--> 0
d1: ℤ^2 --> ℤ^2,
(1, 2)
(0, 2)
d2: ℤ^2 --> 0,
(0, 0)

The cohomology groups are:
[0, ℤ/2ℤ]


In [26]:
Z = ZModule.free(1)
Zsquare = ZModule.free(2)
Zcube = ZModule.free(3)
cochain_complex = CochainComplex(
    modules=[Zsquare, Zcube, Z],
    homomorphisms=[
        Homomorphism([
            [1, 0],
            [0, 1],
            [0, 0]
        ], Zsquare, Zcube),
        Homomorphism([
            [0, 0, 1],
        ], Zcube, Z),
    ]
)
print(cochain_complex)
H = cohomology(cochain_complex)
print("The cohomology groups are:")
print(H)

0 --d0--> ℤ^2 --d1--> ℤ^3 --d2--> ℤ --d3--> 0
d1: ℤ^2 --> ℤ^3,
(1, 0)
(0, 1)
(0, 0)
d2: ℤ^3 --> ℤ,
(0, 0, 1)
d3: ℤ --> 0,
(0,)

The cohomology groups are:
[0, 0, 0]


In [27]:
C0 = ZModule(1, [4])
C1 = ZModule(2, [2])
C2 = ZModule(1, [4])
d1 = Homomorphism((
    (1, 0),
    (2, 0),
    (1, 1),
), C0, C1)
d2 = Homomorphism((
    (-2, 1, 0),
    (0, 0, 0),
), C1, C2)
cochain_complex = CochainComplex(
    modules=[C0, C1, C2],
    homomorphisms=[d1, d2]
)
print(cochain_complex)
H = cohomology(cochain_complex)
print("The cohomology groups are:")
print(H)

0 --d0--> ℤ ⊕ ℤ/4ℤ --d1--> ℤ^2 ⊕ ℤ/2ℤ --d2--> ℤ ⊕ ℤ/4ℤ --d3--> 0
d1: ℤ ⊕ ℤ/4ℤ --> ℤ^2 ⊕ ℤ/2ℤ,
(1, 0)
(2, 0)
(1, 1)
d2: ℤ^2 ⊕ ℤ/2ℤ --> ℤ ⊕ ℤ/4ℤ,
(-2, 1, 0)
(0, 0, 0)
d3: ℤ ⊕ ℤ/4ℤ --> 0,
(0, 0)

The cohomology groups are:
[ℤ/2ℤ, 0, ℤ/4ℤ]


In [28]:
Z20 = ZModule(0,[20])
d1 = Homomorphism(((4,),), Z20, Z20)
d2 = Homomorphism(((10,),), Z20, Z20)
cochain_complex = CochainComplex(
    modules=[Z20, Z20, Z20],
    homomorphisms=[d1, d2]
)
print(cochain_complex)
H = cohomology(cochain_complex)
print("The cohomology groups are:")
print(H)

0 --d0--> ℤ/20ℤ --d1--> ℤ/20ℤ --d2--> ℤ/20ℤ --d3--> 0
d1: ℤ/20ℤ --> ℤ/20ℤ,
(4,)
d2: ℤ/20ℤ --> ℤ/20ℤ,
(10,)
d3: ℤ/20ℤ --> 0,
(0,)

The cohomology groups are:
[ℤ/4ℤ, ℤ/2ℤ, ℤ/10ℤ]
